In [1]:
from utils import listAvailableParameters, get_temperature_forecast, getTempSum_point, virenwc_forecast
import pandas as pd
import numpy as np
import os
from datetime import timedelta
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.environ.get('API_KEY')
southern_finland = [21.0, 59.5, 26.0, 62.0]

latitude = 60.7187936947667
longitude = 26.291561244588

startdate = "2025-06-17"
enddate = "2025-06-20"

In [2]:
# Luodaan 5 satunnaista päivämäärää annetulta aikaväliltä
dates = pd.date_range(start="2025-06-14", end="2025-06-19").to_series().sample(5, random_state=42).sort_values().reset_index(drop=True)

# Ladataan koordinaattidata ja otetaan 5 satunnaista riviä
coords = pd.read_csv('data/reference.csv', usecols=['grid_lon', 'grid_lat'])
sampled_coords = coords.sample(5, random_state=42).reset_index(drop=True)

# Luodaan DataFrame yhdistämällä päivämäärät, koordinaatit ja mittaukset
df = pd.DataFrame({
    "Näytteenottopvm": dates,
    "Mittaus1": np.random.uniform(0, 100, size=5),
    "Mittaus2": np.random.normal(50, 10, size=5),
    "lon": sampled_coords["grid_lon"],
    "lat": sampled_coords["grid_lat"]
})

df

,Näytteenottopvm,Mittaus1,Mittaus2,lon,lat
0,2025-06-14,43.393962,74.412463,25.006091,63.102459
1,2025-06-15,60.725657,65.158723,22.566797,60.611289
2,2025-06-16,61.628978,32.589742,21.909868,63.006943
3,2025-06-18,86.672233,54.547622,24.598910,65.842434
4,2025-06-19,41.554489,55.435932,23.582094,62.401658


In [3]:
# Define model configs
models = [
    (df.copy(), 3, "tempSum_3days"),
    (df.copy(), 6, "tempSum_6days"),
]

# Loop over both models
for model_df, days_ahead, col_name in models:
    model_df[col_name] = None

    for i, row in model_df.iterrows():
        print(f"Query date: {row['Näytteenottopvm']} (T+{days_ahead})")

        start_date = row['Näytteenottopvm']
        expected_end_date = start_date + timedelta(days=days_ahead)

        tempsums = getTempSum_point(
            key=API_KEY,
            lat=row['lat'],
            lon=row['lon'],
            startdate=start_date.strftime('%Y-%m-%d'),
            enddate=(expected_end_date + timedelta(days=1)).strftime('%Y-%m-%d')
        )

        if not tempsums.empty:
            latest_date = tempsums["date"].max()
            latest_tempsum = tempsums.loc[tempsums["date"] == latest_date, "tempSum"].values[0]
            print(f"Latest date in tempsums: {latest_date}, value: {latest_tempsum}")

            if latest_date < expected_end_date.date():
                forecast_start = latest_date + timedelta(days=1)
                forecast_end = expected_end_date.date()

                forecasts = virenwc_forecast(
                    key=API_KEY,
                    lat=row['lat'],
                    lon=row['lon'],
                    startdate=forecast_start.strftime('%Y-%m-%d'),
                    enddate=forecast_end.strftime('%Y-%m-%d')
                )

                # Rename column if needed
                if 'temperature' in forecasts.columns:
                    forecasts = forecasts.rename(columns={'temperature': 'avgTemp'})

                cumulative_tempsum = latest_tempsum
                for _, forecast_row in forecasts.iterrows():
                    avg_temp = forecast_row['avgTemp']
                    if avg_temp > 5:
                        cumulative_tempsum += (avg_temp - 5)

                print(f"Cumulative tempsum for {expected_end_date.date()} calculated as: {cumulative_tempsum}")
                model_df.at[i, col_name] = cumulative_tempsum
            else:
                print(f"Tempsum data already available up to expected end date: {expected_end_date.date()}")
                model_df.at[i, col_name] = latest_tempsum
        else:
            print("⚠️ No data returned for this coordinate/date.")
            model_df.at[i, col_name] = None

    # Save back each processed dataframe under a meaningful variable name
    if days_ahead == 3:
        model_plus_3 = model_df
    elif days_ahead == 6:
        model_plus_6 = model_df

model_plus_3

Query date: 2025-06-14 00:00:00 (T+3)
Latest date in tempsums: 2025-06-17, value: 265.9
Tempsum data already available up to expected end date: 2025-06-17
Query date: 2025-06-15 00:00:00 (T+3)
Latest date in tempsums: 2025-06-18, value: 337.2
Tempsum data already available up to expected end date: 2025-06-18
Query date: 2025-06-16 00:00:00 (T+3)
Latest date in tempsums: 2025-06-18, value: 274.1
Cumulative tempsum for 2025-06-19 calculated as: 282.3
Query date: 2025-06-18 00:00:00 (T+3)
Latest date in tempsums: 2025-06-18, value: 197.8
Cumulative tempsum for 2025-06-21 calculated as: 217.70000000000002
Query date: 2025-06-19 00:00:00 (T+3)
Latest date in tempsums: 2025-06-18, value: 295.3
Cumulative tempsum for 2025-06-22 calculated as: 322.90000000000003
Query date: 2025-06-14 00:00:00 (T+6)
Latest date in tempsums: 2025-06-18, value: 273.9
Cumulative tempsum for 2025-06-20 calculated as: 287.79999999999995
Query date: 2025-06-15 00:00:00 (T+6)
Latest date in tempsums: 2025-06-18, valu

,Näytteenottopvm,Mittaus1,Mittaus2,lon,lat,tempSum_3days
0,2025-06-14,43.393962,74.412463,25.006091,63.102459,265.9
1,2025-06-15,60.725657,65.158723,22.566797,60.611289,337.2
2,2025-06-16,61.628978,32.589742,21.909868,63.006943,282.3
3,2025-06-18,86.672233,54.547622,24.598910,65.842434,217.7
4,2025-06-19,41.554489,55.435932,23.582094,62.401658,322.9


In [4]:
model_plus_6

,Näytteenottopvm,Mittaus1,Mittaus2,lon,lat,tempSum_6days
0,2025-06-14,43.393962,74.412463,25.006091,63.102459,287.8
1,2025-06-15,60.725657,65.158723,22.566797,60.611289,362.8
2,2025-06-16,61.628978,32.589742,21.909868,63.006943,302.2
3,2025-06-18,86.672233,54.547622,24.598910,65.842434,241.5
4,2025-06-19,41.554489,55.435932,23.582094,62.401658,349.4


In [3]:
model_plus_3 = df.copy()
model_plus_3["tempSum_3days"] = None
model_plus_6 = df.copy()
model_plus_6["tempSum_6days"] = None

for i, row in model_plus_3.iterrows():
    print(f"Query date: {row['Näytteenottopvm']}")

    start_date = row['Näytteenottopvm']
    expected_end_date = start_date + timedelta(days=3)

    tempsums = getTempSum_point(
        key=API_KEY,
        lat=row['lat'],
        lon=row['lon'],
        startdate=start_date.strftime('%Y-%m-%d'),
        enddate=(expected_end_date + timedelta(days=1)).strftime('%Y-%m-%d')
    )

    if not tempsums.empty:
        latest_date = tempsums["date"].max()
        latest_tempsum = tempsums.loc[tempsums["date"] == latest_date, "tempSum"].values[0]
        print(f"Latest date in tempsums: {latest_date}, value: {latest_tempsum}")

        if latest_date < expected_end_date.date():
            forecast_start = latest_date + timedelta(days=1)
            forecast_end = expected_end_date.date()

            forecasts = virenwc_forecast(
                key=API_KEY,
                lat=row['lat'],
                lon=row['lon'],
                startdate=forecast_start.strftime('%Y-%m-%d'),
                enddate=forecast_end.strftime('%Y-%m-%d')
            )

            # Rename temperature to avgTemp if needed
            if 'temperature' in forecasts.columns:
                forecasts = forecasts.rename(columns={'temperature': 'avgTemp'})

            cumulative_tempsum = latest_tempsum

            for _, forecast_row in forecasts.iterrows():
                avg_temp = forecast_row['avgTemp']
                if avg_temp > 5:
                    cumulative_tempsum += (avg_temp - 5)

            print(f"Cumulative tempsum for {expected_end_date.date()} calculated as: {cumulative_tempsum}")

            # Save cumulative value to dataframe
            model_plus_3.at[i, "tempSum_3days"] = cumulative_tempsum
        else:
            print(f"Tempsum data already available up to expected end date: {expected_end_date.date()}")
            # Save latest tempsum since it already includes the expected date
            model_plus_3.at[i, "tempSum_3days"] = latest_tempsum
    else:
        print("⚠️ No data returned for this coordinate/date.")
        model_plus_3.at[i, "tempSum_3days"] = None

# same for T+6
for i, row in model_plus_6.iterrows():
    print(f"Query date: {row['Näytteenottopvm']}")

    start_date = row['Näytteenottopvm']
    expected_end_date = start_date + timedelta(days=6)

    tempsums = getTempSum_point(
        key=API_KEY,
        lat=row['lat'],
        lon=row['lon'],
        startdate=start_date.strftime('%Y-%m-%d'),
        enddate=(expected_end_date + timedelta(days=1)).strftime('%Y-%m-%d')
    )

    if not tempsums.empty:
        latest_date = tempsums["date"].max()
        latest_tempsum = tempsums.loc[tempsums["date"] == latest_date, "tempSum"].values[0]
        print(f"Latest date in tempsums: {latest_date}, value: {latest_tempsum}")

        if latest_date < expected_end_date.date():
            forecast_start = latest_date + timedelta(days=1)
            forecast_end = expected_end_date.date()

            forecasts = virenwc_forecast(
                key=API_KEY,
                lat=row['lat'],
                lon=row['lon'],
                startdate=forecast_start.strftime('%Y-%m-%d'),
                enddate=forecast_end.strftime('%Y-%m-%d')
            )

            # Rename temperature to avgTemp if needed
            if 'temperature' in forecasts.columns:
                forecasts = forecasts.rename(columns={'temperature': 'avgTemp'})

            cumulative_tempsum = latest_tempsum

            for _, forecast_row in forecasts.iterrows():
                avg_temp = forecast_row['avgTemp']
                if avg_temp > 5:
                    cumulative_tempsum += (avg_temp - 5)

            print(f"Cumulative tempsum for {expected_end_date.date()} calculated as: {cumulative_tempsum}")

            # Save cumulative value to dataframe
            model_plus_6.at[i, "tempSum_6days"] = cumulative_tempsum
        else:
            print(f"Tempsum data already available up to expected end date: {expected_end_date.date()}")
            # Save latest tempsum since it already includes the expected date
            model_plus_6.at[i, "tempSum_6days"] = latest_tempsum
    else:
        print("⚠️ No data returned for this coordinate/date.")
        model_plus_6.at[i, "tempSum_6days"] = None

model_plus_6

Query date: 2025-06-14 00:00:00
Latest date in tempsums: 2025-06-17, value: 265.9
Tempsum data already available up to expected end date: 2025-06-17
Query date: 2025-06-15 00:00:00
Latest date in tempsums: 2025-06-18, value: 337.2
Tempsum data already available up to expected end date: 2025-06-18
Query date: 2025-06-16 00:00:00
Latest date in tempsums: 2025-06-18, value: 274.1
Cumulative tempsum for 2025-06-19 calculated as: 282.3
Query date: 2025-06-18 00:00:00
Latest date in tempsums: 2025-06-18, value: 197.8
Cumulative tempsum for 2025-06-21 calculated as: 217.70000000000002
Query date: 2025-06-19 00:00:00
Latest date in tempsums: 2025-06-18, value: 295.3
Cumulative tempsum for 2025-06-22 calculated as: 322.90000000000003
Query date: 2025-06-14 00:00:00
Latest date in tempsums: 2025-06-18, value: 273.9
Cumulative tempsum for 2025-06-20 calculated as: 287.59999999999997
Query date: 2025-06-15 00:00:00
Latest date in tempsums: 2025-06-18, value: 337.2
Cumulative tempsum for 2025-06-21

,Näytteenottopvm,Mittaus1,Mittaus2,lon,lat,tempSum_6days
0,2025-06-14,9.245677,54.417135,25.006091,63.102459,287.6
1,2025-06-15,52.254929,70.098193,22.566797,60.611289,362.5
2,2025-06-16,46.396127,44.099881,21.909868,63.006943,302.2
3,2025-06-18,63.512910,55.507481,24.598910,65.842434,241.5
4,2025-06-19,58.309847,57.380566,23.582094,62.401658,349.4


In [4]:
model_plus_3

,Näytteenottopvm,Mittaus1,Mittaus2,lon,lat,tempSum_3days
0,2025-06-14,9.245677,54.417135,25.006091,63.102459,265.9
1,2025-06-15,52.254929,70.098193,22.566797,60.611289,337.2
2,2025-06-16,46.396127,44.099881,21.909868,63.006943,282.3
3,2025-06-18,63.512910,55.507481,24.598910,65.842434,217.7
4,2025-06-19,58.309847,57.380566,23.582094,62.401658,322.9


In [4]:
ts = getTempSum_point(
    key=API_KEY,
    lat=latitude,
    lon=longitude,
    startdate="2025-06-13",
    enddate="2025-06-20"
    )
ts

,date,tempSum
0,2025-06-13,300.7
1,2025-06-14,312.0
2,2025-06-15,327.5
3,2025-06-16,341.0
4,2025-06-17,351.0
5,2025-06-18,359.7


In [5]:
df = virenwc_forecast(
    key=API_KEY,
    lat=latitude,
    lon=longitude,
    startdate="2025-06-17",
    enddate="2025-06-25"
    )
df

,date,avgTemp
0,2025-06-18,15.3
1,2025-06-19,12.7
2,2025-06-20,14.0
3,2025-06-21,12.9
4,2025-06-22,13.0
5,2025-06-23,14.2
6,2025-06-24,15.0
7,2025-06-25,14.9


In [ ]:
params = listAvailableParameters(
    key=API_KEY,
    starttime="2025-06-17", 
    endtime="2025-06-20",
    lat=63.3354490762957,
    lon=23.5715668443889
)

Available observed parameters:
 - meta?observableProperty=forecast&param=BoundaryLayerTurbulence&language=eng
 - meta?observableProperty=forecast&param=CAPE&language=eng
 - meta?observableProperty=forecast&param=CbCloudBase&language=eng
 - meta?observableProperty=forecast&param=CbCloudCover&language=eng
 - meta?observableProperty=forecast&param=CbCloudTop&language=eng
 - meta?observableProperty=forecast&param=CloudBase&language=eng
 - meta?observableProperty=forecast&param=CloudBase2&language=eng
 - meta?observableProperty=forecast&param=CloudTop&language=eng
 - meta?observableProperty=forecast&param=DewPoint&language=eng
 - meta?observableProperty=forecast&param=FlashMultiplicity&language=eng
 - meta?observableProperty=forecast&param=GeopHeight&language=eng
 - meta?observableProperty=forecast&param=GroundTemperature&language=eng
 - meta?observableProperty=forecast&param=HighCloudCover&language=eng
 - meta?observableProperty=forecast&param=Humidity&language=eng
 - meta?observableProper